# FFF step 1: merge prep

In [1]:
%load_ext autoreload
%autoreload 2
import hippo
from os import environ
from pathlib import Path
import json
from mrich import print
import mrich
import shutil
import molparse as mp

## Get BulkDock HIPPO database

In [2]:
target_name = "A71EV2AZ"

In [3]:
animal = hippo.HIPPO(target_name, Path(environ["BULK"])/"TARGETS"/target_name/f"{target_name}.sqlite")

 Creating HIPPO animal

name = A71EV2AZ

db_path = /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/A71EV2AZ/A71EV2AZ.sqlite

DEBUG: hippo.Database.__init__()

DEBUG: Database.path = /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/A71EV2AZ/A71EV2AZ.sqlite

DEBUG: hippo.Database.connect()

DEBUG: sqlite3.version='2.6.0'

 Success  Database connected @ /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/A71EV2AZ/A71EV2AZ.sqlite!

 Success  Initialised animal HIPPO("A71EV2AZ")!

## Define merging hypotheses

In [ ]:
animal.tags.unique

In [ ]:
merging_hypotheses = {
    # nickname: tag,
    "iter1_frags" : "[Other] FFF_iter1_frags",
}

json.dump(merging_hypotheses, open("merging_hypotheses.json", "wt"))

## Create subdirectories

In [ ]:
for nickname in merging_hypotheses:
    (Path("../fragmenstein") / nickname).mkdir(exist_ok=True, parents=True)
    (Path("../knitwork") / nickname).mkdir(exist_ok=True, parents=True)
    (Path("../syndirella") / nickname).mkdir(exist_ok=True, parents=True)

## Knitwork & Fragmenstein inputs

### Create input SDFs & CSVs

In [ ]:
for nickname, tag in merging_hypotheses.items():
    
    poses = animal.poses(tag=tag)
    mrich.h3(nickname)
    mrich.var("poses", poses)

    # knitwork
    out_dir = Path(f"../knitwork/{nickname}")
    poses.to_knitwork(out_dir / f"{nickname}_input.csv", path_root=out_dir, aligned_files_dir="../aligned_files")

    # # fragmenstein
    # poses.write_sdf(out_dir / f"{nickname}_hits.sdf")
    # shutil.copy(out_dir / f"{nickname}_hits.sdf", Path(f"../fragmenstein/{nickname}") / f"{nickname}_hits.sdf")

    # ref_pose = poses[0]
    # mrich.var("ref_pose", ref_pose)
    # shutil.copy(ref_pose.apo_path, Path(f"../fragmenstein/{nickname}"))

### Create protonated PDBs for Knitwork

In [ ]:
for nickname, tag in merging_hypotheses.items():
    poses = animal.poses(tag=tag)
    for pose in poses:
        path = pose.apo_path
        sys = mp.parse(path,verbosity=False)
        sys.add_hydrogens(pH=7)
        subdir = Path("../aligned_files")/pose.name
        subdir.mkdir(exist_ok=True, parents=True)
        sys.write(subdir / str(path.name).replace('_apo-desolv.pdb', '_apo-desolv-Hs.pdb'), verbosity=1)

## Backup and close database

In [ ]:
animal.db.backup()
animal.db.close()

In [4]:
animal.poses['A0229a', 'A0503a', 'A0759a'].to_syndirella("../syndirella/iter1_elabs")

 DISK  Writing ../syndirella/templates...

out_key = iter1_elabs

#poses = 3

Output()

 Warning  C1->P1: "A0229a" has no reference, using self as template!

...

 ERROR  No entry in subsite_tag with subsite_tag_pose=1!

 ERROR  No entry in subsite_tag with subsite_tag_pose=5!

 ERROR  No entry in subsite_tag with subsite_tag_pose=6!

 DISK  Writing ../syndirella/iter1_elabs_syndirella_inspiration_hits.sdf...

,smiles,template,compound_set,hit1
0,COc1ccc2[nH]cc(CN(C)C)c2c1,A0229a,iter1_elabs,A0229a
1,Cc1nc2ccccc2n1CCC(N)=O,A0503a,iter1_elabs,A0503a
2,CC(O)CNCc1ccc2c(c1)OCO2,A0759a,iter1_elabs,A0759a


In [10]:
animal.compounds(smiles="Cc1nc2ccccc2n1CCC(=O)NC[C@H](C)O")

C10 "NCUZRMAXKXZOMY-UHFFFAOYSA-N"

In [15]:
for c_id in [544,554,563,565,814,842,843,848,850,860,876,881,908,973,973,980,318,318,332,411,412,421,527,527,554,557,557,560,560,563,565,570,570,574,814,818,837,837,411,412,10]:
        c = animal.compounds[c_id]
        p = c.best_placed_pose
        print(",".join(p.inspirations.names), p.reference.name)

A0229a,A0503a A0503a

A0229a,A0503a A0503a

A0229a,A0503a A0503a

A0229a,A0503a A0503a

A0503a,A0759a A0759a

...

A0503a,A0759a A0759a

A0503a,A0759a A0759a

A0503a,A0759a A0503a

A0503a,A0759a A0503a

A0503a,A0759a A0759a